In [1]:
pip install gspread pandas scikit-learn oauth2client

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install gspread oauth2client


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error, r2_score
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to load and process data
def load_and_process_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Function to update Google Sheets
def update_google_sheet(sheet_name, data):
    # Define the scope and authenticate
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
    client = gspread.authorize(creds)

    # Open the Google Sheet
    sheet = client.open(sheet_name).sheet1

    # Clear existing data
    sheet.clear()

    # Update with new data
    sheet.update([data.columns.values.tolist()] + data.values.tolist())

# Main loop for real-time analysis
file_path = 'roomdata.csv'
previous_data = None

while True:
    # Load the data from the CSV file
    current_data = load_and_process_data(file_path)

    # Check if the data has changed
    if not current_data.equals(previous_data):
        print("Data has changed, performing analysis...")

        # Prepare the data for classification
        X_classification = current_data[['People', 'Temperature', 'Humidity']]
        y_classification = current_data['Room Occupied'].astype(int)

        # Split the data into training and testing sets for classification
        X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
            X_classification, y_classification, test_size=0.2, random_state=42
        )

        # Create and train the Random Forest Classifier
        rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_classifier.fit(X_train_class, y_train_class)

        # Make predictions
        y_pred_class = rf_classifier.predict(X_test_class)

        # Evaluate the classifier
        print("Classification Report:")
        print(classification_report(y_test_class, y_pred_class))

        # Prepare the data for regression
        X_regression = current_data[['People', 'Temperature', 'Humidity']]
        y_regression = current_data['Time Occupied']

        # Split the data into training and testing sets for regression
        X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
            X_regression, y_regression, test_size=0.2, random_state=42
        )

        # Create and train the Random Forest Regressor
        rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_regressor.fit(X_train_reg, y_train_reg)

        # Make predictions
        y_pred_reg = rf_regressor.predict(X_test_reg)

        # Evaluate the regressor
        print("Regression Metrics:")
        print(f"Mean Squared Error: {mean_squared_error(y_test_reg, y_pred_reg)}")
        print(f"R^2 Score: {r2_score(y_test_reg, y_pred_reg)}")

        # Update Google Sheets with the current data
        update_google_sheet('CIA1comp2', current_data)

        # Update previous data
        previous_data = current_data

    # Wait for a specified time before checking again
    time.sleep(5)  # Check for updates every 5 seconds

Data has changed, performing analysis...


KeyError: "None of [Index(['People', 'Temperature', 'Humidity'], dtype='object')] are in the [columns]"

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the data from the CSV file
df = pd.read_csv("roomdata.csv")

# Print the DataFrame and its columns for debugging
print("DataFrame Columns:", df.columns.tolist())
print(df.head())  # Print the first few rows of the DataFrame

# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Check if the required columns exist
required_columns = ['People', 'Temperature', 'Humidity']
for col in required_columns:
    if col not in df.columns:
        print(f"Column '{col}' not found in DataFrame.")
        # Handle the missing column as needed (e.g., skip, raise an error, etc.)

# Assuming 'Room Occupancy' is the target variable and the rest are features
X = df[required_columns]  # Features
y = df['Room Occupancy']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

DataFrame Columns: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
   Room Occupancy   People   Temperature   Humidity   Time Occupied
0               0        0          27.6       78.0               0
1               0        0          27.6       78.0               0
2               0        0          27.6       78.0               0
3               0        0          27.6       78.0               0
4               0        0          27.6       78.0               0
Mean Squared Error: 0.0


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

# Load the data from the CSV file
df = pd.read_csv("roomdata.csv")

# Print the DataFrame and its columns for debugging
print("DataFrame Columns:", df.columns.tolist())
print(df.head())  # Print the first few rows of the DataFrame

# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Check if the required columns exist
required_columns_classification = ['People', 'Temperature', 'Humidity']
required_columns_regression = ['People', 'Temperature', 'Humidity', 'Time Occupied']

for col in required_columns_classification + required_columns_regression:
    if col not in df.columns:
        print(f"Column '{col}' not found in DataFrame.")
        # Handle the missing column as needed (e.g., skip, raise an error, etc.)

# Prepare data for classification
X_classification = df[required_columns_classification]  # Features for classification
y_classification = df['Room Occupancy']  # Target variable for classification

# Split the data into training and testing sets for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

# Create and train the Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_class, y_train_class)

# Make predictions for classification
y_pred_class = classifier.predict(X_test_class)

# Evaluate the classification model
accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Classification Accuracy: {accuracy}")

# Prepare data for regression
X_regression = df[required_columns_regression[:-1]]  # Features for regression (excluding 'Time Occupied')
y_regression = df['Time Occupied']  # Target variable for regression

# Split the data into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Create and train the Random Forest Regressor
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_reg, y_train_reg)

# Make predictions for regression
y_pred_reg = regressor.predict(X_test_reg)

# Evaluate the regression model
mse = mean_squared_error(y_test_reg, y_pred_reg)
print(f"Regression Mean Squared Error: {mse}")

DataFrame Columns: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
   Room Occupancy   People   Temperature   Humidity   Time Occupied
0               0        0          27.6       78.0               0
1               0        0          27.6       78.0               0
2               0        0          27.6       78.0               0
3               0        0          27.6       78.0               0
4               0        0          27.6       78.0               0
Classification Accuracy: 1.0
Regression Mean Squared Error: 2.8332254553251683


In [6]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import os

# Initialize variables
csv_file_path = "roomdata.csv"
last_modified_time = os.path.getmtime(csv_file_path)

# Load initial data from the CSV file
df = pd.read_csv(csv_file_path)

# Prepare data for classification
X_classification = df[['People', 'Temperature', 'Humidity']]
y_classification = df['Room Occupancy']

# Split the data into training and testing sets for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

# Create and train the Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_class, y_train_class)

# Prepare data for regression
X_regression = df[['People', 'Temperature', 'Humidity']]
y_regression = df['Time Occupied']

# Split the data into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Create and train the Random Forest Regressor
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_reg, y_train_reg)

# Function to monitor CSV file for updates
def monitor_csv():
    global last_modified_time
    while True:
        current_modified_time = os.path.getmtime(csv_file_path)
        if current_modified_time != last_modified_time:
            print("CSV file updated. Reloading data...")
            last_modified_time = current_modified_time
            
            # Load the updated data from the CSV file
            df = pd.read_csv(csv_file_path)

            # Prepare data for classification
            X_classification = df[['People', 'Temperature', 'Humidity']]
            y_classification = df['Room Occupancy']

            # Update the classifier
            classifier.fit(X_classification, y_classification)

            # Prepare data for regression
            X_regression = df[['People', 'Temperature', 'Humidity']]
            y_regression = df['Time Occupied']

            # Update the regressor
            regressor.fit(X_regression, y_regression)

        time.sleep(5)  # Check for updates every 5 seconds

# Start monitoring the CSV file
monitor_csv()

KeyError: "None of [Index(['People', 'Temperature', 'Humidity'], dtype='object')] are in the [columns]"

In [1]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import os

# Initialize variables
csv_file_path = "roomdata.csv"
last_modified_time = os.path.getmtime(csv_file_path)

# Function to load and prepare data
def load_data():
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()  # Strip whitespace from column names
    return df

# Load initial data from the CSV file
df = load_data()

# Check if required columns exist
required_columns = ['People', 'Temperature', 'Humidity', 'Room Occupancy', 'Time Occupied']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"CSV file must contain the following columns: {required_columns}")

# Prepare data for classification
X_classification = df[['People', 'Temperature', 'Humidity']]
y_classification = df['Room Occupancy']

# Split the data into training and testing sets for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

# Create and train the Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_class, y_train_class)

# Prepare data for regression
X_regression = df[['People', 'Temperature', 'Humidity']]
y_regression = df['Time Occupied']

# Split the data into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Create and train the Random Forest Regressor
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_reg, y_train_reg)

# Function to monitor CSV file for updates
def monitor_csv():
    global last_modified_time
    while True:
        current_modified_time = os.path.getmtime(csv_file_path)
        if current_modified_time != last_modified_time:
            print("CSV file updated. Reloading data...")
            last_modified_time = current_modified_time
            
            # Load the updated data from the CSV file
            df = load_data()

            # Check if required columns exist
            if not all(col in df.columns for col in required_columns):
                print(f"CSV file must contain the following columns: {required_columns}")
                continue  # Skip this iteration if columns are missing

            # Prepare data for classification
            X_classification = df[['People', 'Temperature', 'Humidity']]
            y_classification = df['Room Occupancy']

            # Update the classifier
            classifier.fit(X_classification, y_classification)

            # Prepare data for regression
            X_regression = df[['People', 'Temperature', 'Humidity']]
            y_regression = df['Time Occupied']

            # Update the regressor
            regressor.fit(X_regression, y_regression)

        time.sleep(5)  # Check for updates every 5 seconds

# Start monitoring the CSV file
monitor_csv()

CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...
CSV file updated. Reloading data...



KeyboardInterrupt



In [2]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import os

# Initialize variables
csv_file_path = "roomdata.csv"
last_modified_time = os.path.getmtime(csv_file_path)

# Function to load and prepare data
def load_data():
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.strip()  # Strip whitespace from column names
    return df

# Load initial data from the CSV file
df = load_data()

# Check if required columns exist
required_columns = ['People', 'Temperature', 'Humidity', 'Room Occupancy', 'Time Occupied']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"CSV file must contain the following columns: {required_columns}")

# Prepare data for classification
X_classification = df[['People', 'Temperature', 'Humidity']]
y_classification = df['Room Occupancy']

# Split the data into training and testing sets for classification
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=42
)

# Create and train the Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_class, y_train_class)

# Prepare data for regression
X_regression = df[['People', 'Temperature', 'Humidity']]
y_regression = df['Time Occupied']

# Split the data into training and testing sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=42
)

# Create and train the Random Forest Regressor
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_reg, y_train_reg)

# Function to monitor CSV file for updates
def monitor_csv():
    global last_modified_time
    while True:
        current_modified_time = os.path.getmtime(csv_file_path)
        if current_modified_time != last_modified_time:
            print("CSV file updated. Reloading data...")
            last_modified_time = current_modified_time
            
            # Load the updated data from the CSV file
            df = load_data()
            print("Updated DataFrame:")
            print(df)  # Print the updated DataFrame

            # Check if required columns exist
            if not all(col in df.columns for col in required_columns):
                print(f"CSV file must contain the following columns: {required_columns}")
                continue  # Skip this iteration if columns are missing

            # Prepare data for classification
            X_classification = df[['People', 'Temperature', 'Humidity']]
            y_classification = df['Room Occupancy']

            # Update the classifier
            classifier.fit(X_classification, y_classification)

            # Prepare data for regression
            X_regression = df[['People', 'Temperature', 'Humidity']]
            y_regression = df['Time Occupied']

            # Update the regressor
            regressor.fit(X_regression, y_regression)

            # Print confirmation of model updates
            print("Models updated with new data.")

        time.sleep(5)  # Check for updates every 5 seconds

# Start monitoring the CSV file
monitor_csv()

CSV file updated. Reloading data...
Updated DataFrame:
     Room Occupancy  People  Temperature  Humidity  Time Occupied
0                 0       0         27.6      78.0              0
1                 0       0         27.6      78.0              0
2                 0       0         27.6      78.0              0
3                 0       0         27.6      78.0              0
4                 0       0         27.6      78.0              0
..              ...     ...          ...       ...            ...
221               0       0         28.0      77.0              0
222               0       0         28.0      77.0              0
223               0       0         28.0      77.0              0
224               0       0         28.0      77.0              0
225               0       0         28.0      77.0              0

[226 rows x 5 columns]
Models updated with new data.
CSV file updated. Reloading data...
Updated DataFrame:
     Room Occupancy  People  Temperature  Hu

KeyboardInterrupt: 

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

# Function to read and preprocess data
def read_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    
    # Print the columns to debug
    print("Columns in the DataFrame:", df.columns.tolist())
    
    # Trim whitespace from column names
    df.columns = df.columns.str.strip()
    
    # Check if required columns exist
    required_columns = ['Room Occupancy', 'People', 'Temperature', 'Humidity', 'Time Occupied']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' is not in the DataFrame.")
    
    # Encode 'Room Occupancy' if it's categorical
    label_encoder = LabelEncoder()
    df['Room Occupancy'] = label_encoder.fit_transform(df['Room Occupancy'])
    
    # Features and targets
    X = df[['People', 'Temperature', 'Humidity']]  # Features
    y_class = df['Room Occupancy']  # Classification target
    y_reg = df['Time Occupied']  # Regression target
    
    return X, y_class, y_reg

# Function to train models
def train_models(X, y_class, y_reg):
    X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(X, y_class, y_reg, test_size=0.2, random_state=42)
    
    # Train Random Forest Classifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_class_train)
    
    # Train Random Forest Regressor
    regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    regressor.fit(X_train, y_reg_train)
    
    return classifier, regressor, X_test, y_class_test, y_reg_test

# Function to make predictions
def make_predictions(classifier, regressor, X_test):
    class_predictions = classifier.predict(X_test)
    reg_predictions = regressor.predict(X_test)
    return class_predictions, reg_predictions

# Main loop to read data in real-time
file_path = 'roomdata.csv'  # Path to your CSV file
while True:
    try:
        X, y_class, y_reg = read_and_preprocess_data(file_path)
        classifier, regressor, X_test, y_class_test, y_reg_test = train_models(X, y_class, y_reg)
        
        class_predictions, reg_predictions = make_predictions(classifier, regressor, X_test)
        
        # Evaluate models
        print(f'Classification Accuracy: {accuracy_score(y_class_test, class_predictions)}')
        print(f'Regression MSE: {mean_squared_error(y_reg_test, reg_predictions)}')
        
        # Provide a random test sample
        random_index = np.random.randint(0, len(X_test))
        print(f'Random Test Sample: {X_test.iloc[random_index]}')
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
    time.sleep(10)  # Wait for 10 seconds before reading the file again

Columns in the DataFrame: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
Classification Accuracy: 1.0
Regression MSE: 0.0
Random Test Sample: People          0.0
Temperature    28.0
Humidity       78.0
Name: 135, dtype: float64
Columns in the DataFrame: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
Classification Accuracy: 1.0
Regression MSE: 0.0
Random Test Sample: People          0.0
Temperature    28.0
Humidity       77.0
Name: 51, dtype: float64
Columns in the DataFrame: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
Classification Accuracy: 1.0
Regression MSE: 0.0
Random Test Sample: People          0.0
Temperature    28.0
Humidity       77.0
Name: 9, dtype: float64
Columns in the DataFrame: ['Room Occupancy', ' People', ' Temperature', ' Humidity', ' Time Occupied']
Classification Accuracy: 1.0
Regression MSE: 0.0
Random Test Sample: People          0.0
Temperature    28.0
Humidity       78

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import time
import os
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

# Function to read and preprocess data
def read_and_preprocess_data(file_path):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None, None, None
    
    df = pd.read_csv(file_path)
    
    # Print the columns to debug
    print("Columns in the DataFrame:", df.columns.tolist())
    
    # Trim whitespace from column names
    df.columns = df.columns.str.strip()
    
    # Check if required columns exist
    required_columns = ['Room Occupancy', 'People', 'Temperature', 'Humidity', 'Time Occupied']
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' is not in the DataFrame.")
    
    # Encode 'Room Occupancy' if it's categorical
    label_encoder = LabelEncoder()
    df['Room Occupancy'] = label_encoder.fit_transform(df['Room Occupancy'])
    
    # Features and targets
    X = df[['People', 'Temperature', 'Humidity']]  # Features
    y_class = df['Room Occupancy']  # Classification target
    y_reg = df['Time Occupied']  # Regression target
    
    return X, y_class, y_reg

# Function to train models
def train_models(X, y_class, y_reg):
    X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(X, y_class, y_reg, test_size=0.2, random_state=42)
    
    # Train Random Forest Classifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_class_train)
    
    # Train Random Forest Regressor
    regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    regressor.fit(X_train, y_reg_train)
    
    return classifier, regressor, X_test, y_class_test, y_reg_test, X_train, y_class_train, y_reg_train

# Function to make predictions
def make_predictions(classifier, regressor, X):
    class_predictions = classifier.predict(X)
    reg_predictions = regressor.predict(X)
    return class_predictions, reg_predictions

# Main loop to read data in real-time
file_path = 'data.csv'  # Path to your CSV file
classifier, regressor = None, None

# Initial training
try:
    X, y_class, y_reg = read_and_preprocess_data(file_path)
    if X is not None and y_class is not None and y_reg is not None:
        classifier, regressor, X_test, y_class_test, y_reg_test, X_train, y_class_train, y_reg_train = train_models(X, y_class, y_reg)
        
        # Provide a random test sample from the training data
        random_index = np.random.randint(0, len(X_train))
        print(f'Random Test Sample from Training Data: {X_train.iloc[random_index].to_dict()}')
    else:
        print("Initial data loading failed. Exiting.")
        exit(1)
        
except Exception as e:
    print(f"An error occurred during initial training: {e}")
    exit(1)

# Real-time data processing loop
while True:
    try:
        print("Reading real-time data...")
        X_real_time, _, _ = read_and_preprocess_data(file_path)
        
        if X_real_time is not None:
            # Make predictions on the real-time data
            class_predictions, reg_predictions = make_predictions(classifier, regressor, X_real_time)
            
            # Print predictions
            print(f'Predictions for Room Occupancy: {class_predictions}')
            print(f'Predictions for Time Occupied: {reg_predictions}')
        else:
            print("Real-time data loading failed.")
        
    except Exception as e:
        print(f"An error occurred during real-time processing: {e}")
    
    time.sleep(5)  # Wait for 5 seconds before reading the file again